#### Задание:  
Проверить гипотезу, что боевики снятые в период с 1990 по 1990 год имеют средний рейтинг выше по сравнению с боевиками снятыми в период с 1980 по 1989 и в период c 2000 по 2009.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

Датасет взят с сайта imdb.com

Подготовим и исследуем данные для проверки гипотезы.

In [2]:
data_basics = pd.read_csv('input/data_basics.tsv', sep='\t')
ratings = pd.read_csv('input/data_ratings.tsv', sep='\t')

In [3]:
data_basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


In [4]:
ratings.head(3)

,tconst,averageRating,numVotes
0,tt0000001,5.8,1504
1,tt0000002,6.3,183
2,tt0000003,6.6,1150


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 938096 entries, 0 to 938095
Data columns (total 3 columns):
tconst           938096 non-null object
averageRating    938096 non-null float64
numVotes         938096 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 21.5+ MB


Из этих файла "data_basics.tsv" нам будут необходимы столбцы:  
startYear (YYYY) - представляет год выпуска заголовка. В случае с сериалом это год начала сериала  
endYear (YYYY) - год окончания сериала. ‘\N’ для всех других типов заголовков  
genres (строковый массив) - включает до трех жанров, связанных с заголовком  

Из этих файла "data_ratings.tsv" нам будет необходим столбец:  
averageRating - средневзвешенная оценка всех отдельных пользовательских рейтингов  

In [6]:
data_basics.shape, ratings.shape

((5892546, 9), (938096, 3))

Разница в количестве строк, в двух файлах говорит о том, что не у всех фильмов есть рейтинги.  

Объединим эти файлы только по фильмам, которые есть в обоих файлах, чтобы было удобно обрабатывать данные.

In [7]:
data = pd.merge(data_basics, ratings, how='inner', on='tconst')

In [8]:
data.shape

(938096, 11)

Посмотрим количество уникальных записей в столбце genres и посмотрим, где встречается слово 'Action'

In [9]:
data['genres'].nunique()

1938

In [10]:
data['genres'] = data['genres'].astype(str)
uniq = data['genres'].unique()

lst_action = []
for i in uniq:
    if 'Action' in i:
        lst_action.append(i)
len(lst_action)

257

In [11]:
lst_action[0:5]

['Action,Documentary,Short',
 'Action,Comedy,Documentary',
 'Action,Adventure,Comedy',
 'Action,Crime,Short',
 'Action,Short']

Т.к. боевики в чистом виде встречаются редко, в основном это комедийные боевики, с приключениями и т.д.
Найдем строки, в которых есть слово 'Action'

In [12]:
data['find_action'] = data['genres'].str.contains('Action')

In [13]:
data.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,find_action
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.8,1504,False
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.3,183,False
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.6,1150,False


Посмотрим на минимальное и максимальное значение рейтинга, чтобы убедиться, что у нас нет выбросов

In [14]:
data['averageRating'].min()

1.0

In [15]:
data['averageRating'].max()

10.0

Напишем функцию, которая будет нам возвращать список рейтингов фильмов в заданном диапазоне годов.   
Мы будем искать строки подходящие нам по 4 параметрам:  
1. что год выпуска находится в необходимом диапазоне
2. что год завершения у него '/N' - это говорит о том, что это фильм
3. Ищем встолбце find_action, что этот фильм относится к боевикам ('Action')

In [16]:
def lst_ratings(start_of_range, end_of_range): 
    year = []
    for i in range(start_of_range, end_of_range+1):
        year.append(str(i))
        
    return(data.loc[(data['startYear'].isin(year)) 
                     & (data['endYear'] == '\\N') 
                     & (data['find_action'] == True), 'averageRating'].values)

In [17]:
ratins_action_1980_1989 =  lst_ratings(1980, 1989)
ratins_action_1990_1999 =  lst_ratings(1990, 1999)
ratins_action_2000_2009 =  lst_ratings(2000, 2009)

Посмотрим средние значения рейтингов фильмов по разным периодам

In [18]:
ratins_action_1980_1989.mean(), ratins_action_1990_1999.mean(), ratins_action_2000_2009.mean()

(6.569246241795469, 6.714620950394368, 6.940552581333534)

Проведем дисперсионый анализ, проверим гипотезу, что средний рейтинг этих фильмов действительно отличается, а затем сравним их средние оценки, если она подтвердится.

Выдвинем гипотезу $H_0$, что средняя оценка у фильмов всех трех периодов одинаковая.

Наблюдаемые данные обозначим $y_{ij}$, где $i$ — индекс уровня ($i = 1, 2, \dots, k$), $j$ — индекс наблюдения на $i$-м уровне ($j = 1, 2, \dots, n_i$). Здесь $n_i$ - число наблюдений на $i$-м уровне. Таким образом, 

$$y_i = \{ y_{i1}, \dots, y_{i n_i} \}$$

для $i = 1, 2, \dots, k$. Соответственно, $\overline{y_i}$ - среднее значение на выборке $y_i$. Обозначим через $n$ общее число наблюдений:

$$n = \displaystyle\sum_{j=1}^{k} n_i.$$

Среднее от всей выборки:

$$\overline{y} = \dfrac{1}{n} \displaystyle\sum_{i=1}^k \displaystyle\sum_{j=1}^{n_i} y_{ij}.$$

Сумма квадратов отклонений наблюдений от общего среднего:

$$S^2 = \displaystyle\sum_{i=1}^{k} \displaystyle\sum_{j=1}^{n_i} ({y}_{ij} - \overline{y})^2.$$

Эту сумму можно разбить на сумму квадратов отклонений средних групповых значений от общего среднего значения $\overline{y}$:

$$S_F^2 = \displaystyle\sum_{i=1}^k (\overline{y}_i - \overline{y})^2 n_i$$

и остаточную сумму квадратов отклонений:

$$S_{res}^{2} = \displaystyle\sum_{i=1}^k \displaystyle\sum_{j=1}^{n_i} (y_{ij} - \overline{y}_i)^2.$$

Для этих значений должно быть справедливо равенство

$$S^2 = S_F^2 + S_{res}^2.$$

По этим значениям можно вычислить соответствующие несмещённые оценки дисперсий:

$$\sigma^2 = \dfrac{S^2}{n - 1}, \: \sigma_F^2 = \dfrac{S_F^2}{k - 1}, \: \sigma_{res}^2 = \dfrac{S_{res}^2}{n - k}.$$

Для проверки гипотезы $H_0$ запишем статистику

$$T = \dfrac{\sigma_F^2}{\sigma_{res}^2}.$$

В предположении верности гипотезы $H_0$ статистика $T$ имеет распределение Фишера с параметрами $k_1 = k - 1$, $k_2 = n - k$. Выберем уровнь значимости $\alpha$ = 0.05 и рассчитаем критическое значение $F_{crit}$, равное квантилю порядка $1 - \alpha$ для распределения $F(k_1, k_2)$. Если $T > F_{crit}$, то гипотеза $H_0$ отвергается.

In [19]:
n1 = ratins_action_1980_1989.shape[0]
n2 = ratins_action_1990_1999.shape[0]
n3 = ratins_action_2000_2009.shape[0]

n1, n2, n3

(9446, 15341, 26711)

Выборочные средние по каждой группе:

In [20]:
ratins_action_1980_1989_mean = ratins_action_1980_1989.mean()
ratins_action_1990_1999_mean = ratins_action_1990_1999.mean()
ratins_action_2000_2009_mean = ratins_action_2000_2009.mean()

ratins_action_1980_1989_mean, ratins_action_1990_1999_mean, ratins_action_2000_2009_mean

(6.569246241795469, 6.714620950394368, 6.940552581333534)

Соберём все значения рейтингов фильмов в один массив и найдем его среднее значение

In [21]:
ratins_action_all = np.concatenate([ratins_action_1980_1989, ratins_action_1990_1999, ratins_action_2000_2009])

ratins_action_all.shape[0]

51498

In [22]:
ratins_action_all_mean = ratins_action_all.mean()
ratins_action_all_mean

6.805141947260088

Найдём значения $S_F$ и $S_{res}$:

In [23]:
S2_F = n1 * (ratins_action_1980_1989_mean - ratins_action_all_mean) ** 2 + n2 * \
            (ratins_action_1990_1999_mean - ratins_action_all_mean) ** 2 + n3 * \
            (ratins_action_2000_2009_mean - ratins_action_all_mean) ** 2

S2_res = ((ratins_action_1980_1989 - ratins_action_1980_1989_mean) ** 2).sum() + \
            ((ratins_action_1990_1999 - ratins_action_1990_1999_mean) ** 2).sum() + \
            ((ratins_action_2000_2009 - ratins_action_2000_2009_mean) ** 2).sum()

S2_F, S2_res

(1141.1184143978912, 97836.51999796764)

Проверим выполнение равенства $S^2 = S_F^2 + S_{res}^2:$ 

In [24]:
S2_F + S2_res == ((ratins_action_all - ratins_action_all_mean) ** 2).sum()

False

In [25]:
S2_F + S2_res, ((ratins_action_all - ratins_action_all_mean) ** 2).sum()

(98977.63841236553, 98977.63841236552)

Равенство $S^2 = S_F^2 + S_{res}^2$ не выполнилось, т.к. у компьютера есть погрешность в вычислениях, и это видно, когда мы вывели значения на экран.

Найдем оценки дисперсий:

In [26]:
k = 3
n = n1 + n2 + n3

k1 = k - 1
k2 = n - k

sigma2_F = S2_F / k1
sigma2_res = S2_res / k2

sigma2_F, sigma2_res

(570.5592071989456, 1.8999227109033427)

Найдем значение статистики $T$:

In [27]:
T = sigma2_F / sigma2_res
T

300.3065355893693

Зафиксируем уровень значимости $\alpha = 0.05$. Для него найдём критическое значение $F_{crit}$:

In [28]:
alpha = 0.05

F_crit = stats.f.ppf(1 - alpha, k1, k2)
F_crit

2.995906557653272

Видим, что $T > F_{crit}$, поэтому заключаем, что отличие средних оценок за фильмы разных периодов действительно является статистически значимым.`

Снова выведем средние оценки по фильмам для разных периодов

In [29]:
ratins_action_1980_1989_mean, ratins_action_1990_1999_mean, ratins_action_2000_2009_mean

(6.569246241795469, 6.714620950394368, 6.940552581333534)

#### Вывод:
И мы видим, что наша гипотеза, что боевики снятые в период с 1990 по 1999 год имеют средний рейтинг выше по сравнению с боевиками снятыми в период с 1980 по 1989 и в период 2000-2009 не подтвердилась. Т.к. средний рейтинг боевиков снятых в период 2000-2009 больше, чем средний рейтинг боевиков снятых в период 1990-1999.